In [1]:
# 기본 라이브러리 불러오기
import pandas as pd
from tqdm.notebook import tqdm  # 반복문의 진행 상태를 표시하는 라이브러리
import matplotlib.pyplot as plt
import datasets
from datasets import Dataset, DatasetDict
import numpy as np

/home/sdf1ai810/miniconda3/envs/module01/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch import cuda

# GPU 사용
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
train_df = pd.read_csv('train.hate.csv')

In [4]:
train_df

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate
...,...,...
7891,힘내세요~ 응원합니다!!,none
7892,힘내세요~~삼가 고인의 명복을 빕니다..,none
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,none
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,none


In [5]:
label2idx= {'none': 0, 'offensive': 1, 'hate': 2} 
train_df['label']= train_df['label'].map(lambda x: label2idx[x])
train_df.head(10)

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,2
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",2
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,2
5,10+8 진짜 이승기랑 비교된다,0
6,"100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, ...",2
7,10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음,0
8,10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐,1
9,10년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름...,1


In [6]:
import datasets
datasets.__version__

'2.21.0'

In [7]:
train_dataset= Dataset.from_pandas(train_df) 


In [8]:
val_df= pd.read_csv('dev.hate.csv')

In [9]:
val_df

,comments,label
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,none
1,지현우 나쁜놈,offensive
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,hate
3,설마 ㅈ 현정 작가 아니지??,hate
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,offensive
...,...,...
466,지현우 범죄 저지르지 않았나요?,offensive
467,여자인생 망칠 일 있나 ㅋㅋ,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate


In [10]:
val_dataset= Dataset.from_pandas(val_df)

In [11]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
import torch

In [12]:
tokenizer= BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')

/home/sdf1ai810/miniconda3/envs/module01/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
def tokenize_function(examples): 
    return tokenizer(examples['comments'], padding= 'max_length', truncation= True) 


In [14]:
train_dataset= train_dataset.map(tokenize_function, batched= True) 

val_dataset= val_dataset.map(tokenize_function, batched= True) 


Map: 100%|██████████| 471/471 [00:00<00:00, 3917.77 examples/s]


In [15]:
train_dataset.set_format(type= 'torch', columns= ['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type= 'torch', columns= ['input_ids', 'attention_mask', 'label'])

In [16]:
model= BertForSequenceClassification.from_pretrained('google-bert/bert-base-multilingual-cased', num_labels= 3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
training_args= TrainingArguments(output_dir= './results', 
                                 eval_strategy= 'epoch', 
                                 per_device_train_batch_size= 8, 
                                 per_device_eval_batch_size= 8, 
                                 num_train_epochs= 4, 
                                 weight_decay= 0.01, 
                                 logging_dir= './logs', 
                                 logging_steps= 50 
)

In [18]:
trainer= Trainer(model= model, 
                 args= training_args, 
                 train_dataset= train_dataset, 
                 eval_dataset= val_dataset)

In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gnorm42 (gnorm42-gjaischool). Use `wandb login --relogin` to force relogin


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
